<a href="https://colab.research.google.com/github/JJay1010/K-software_-/blob/main/UnggyuLee/q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
import json
from PIL import Image
import re
import numpy as np
import tensorflow
#데이터 읽기import glob
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import keras
from keras import Sequential
from keras.layers import Conv2D, MaxPool2D
from keras.layers import Flatten, Dense, Dropout
import keras
import matplotlib.pyplot as plt

In [ ]:

#rootDir = 'C:/Users/lee/Desktop/swbootcamp/p4/유형별 두피 이미지/Training/'
rootDir = 'E:/q4/유형별 두피 이미지/Training/'

labelDirs = glob.glob(rootDir+'**/*.json',recursive=True)
#imgDirs = glob.glob(rootDir+'**/*.jpg',recursive=True)

imgs =[]
labels = []
width = 150
height =150
def decodeStatus(encodedStatus):
    s0=re.compile('000000')
    s1=re.compile('[123]00000')
    s2=re.compile('0[123]0000')
    s3=re.compile('[0123]0[123]000')
    s4=re.compile('[0123][123][123]000')
    s5=re.compile('[0123][0123]0[123][0123]0')
    s6=re.compile('[0123][0123]00[123]0')
    s7=re.compile('00000[123]')
    if(s0.match(encodedStatus)):
        return 0
    elif(s1.match(encodedStatus)):
        return 1
    elif(s2.match(encodedStatus)):
        return 2
    elif(s3.match(encodedStatus)):
        return 3
    elif(s4.match(encodedStatus)):
        return 4
    elif(s5.match(encodedStatus)):
        return 5
    elif(s6.match(encodedStatus)):
        return 6
    elif(s7.match(encodedStatus)):
        return 7
    else:
        #print(encodedStatus)
        #print("abnormal pattern")
        return -1

  

라벨링 방식
        val1    val2       val3     val4        val5    val6  
순서 : 미세각질,피지과다,모낭사이홍반,모낭홍반/농포,비듬,탈모  
(01) <-상관없음

양호 : 000000                                   ->regex(000000)  
건성 : 100000                                   ->regex(100000)  
지성 : 010000                                   ->regex(010000)  
민감성 : (01)01000                              ->regex([01]01000)  
지루성 : (01)11000                              ->regex([01]11000)  
염증성 : (01)(01)01(01)0                        ->regex([01][01]01[01]0)   
비듬성 : (01)(01)0010                           ->regex([01][01]0010)  
탈모성 : 000001                                 ->regex(000001)

상태값 매핑 : 양호0 건성1 지성2 민감성3 지루성4 염증성5 비듬성6 탈모성7 그외-1

내일할일
json에서 value 1~ value6 값을 읽어서 어떤 상태인지 판단 후 전처리
예) 이미지 1, 양호 와 같은 식으로 이미지와 라벨 매칭


2일차
문제 : 너무 많은 데이터를 메모리에 올릴 수가 없다.  
generator을 쓰면 가능

generator 써도 노트북으로는 너무 많은 시간 소모
데이터셋을 줄일 방법을 생각해본다.  
큰 batch 사이즈와 gpu사용으로 해결

In [ ]:

#모델 빌드
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, 3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(9,activation='softmax')
])
model.compile(loss="categorical_crossentropy", optimizer="adam")
model.summary()
length = 0
def gen():
    for lDir in labelDirs:
        with open(lDir,'r',encoding='UTF8') as file:
                data = json.load(file)
                #print(data)
                #print(data['value_1']+data['value_2']+data['value_3']+data['value_4']+data['value_5']+data['value_6'])
                encodedStatus = data['value_1']+data['value_2']+data['value_3']+data['value_4']+data['value_5']+data['value_6']
                
        
                imgDir = (lDir.replace('라벨','원천')).replace('json','jpg')
                #print(imgDir)
                imgData = np.array(Image.open(imgDir).resize((width,height)).convert('RGB'))
                imgData = imgData/255 
               
            

                label = decodeStatus(encodedStatus)
                label=np.array(label)
                label=keras.utils.to_categorical(label,9)
                label=label.astype(np.int8)
               

                
                yield(imgData,label)
              
d = tf.data.Dataset.from_generator(gen,(tf.int16,9),((width,height,3),(9)))
d = d.shuffle(700).batch(700)
with tf.device("/device:GPU:0"):
    history = model.fit(d,epochs=3)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 34, 34, 64)        36928     
                                                                 
 flatten_1 (Flatten)         (None, 73984)            

In [ ]:
model.save('E:/q4/temp.h5')

모델 검증  

In [ ]:
loadedModel = tf.keras.models.load_model('E:/q4/temp.h5')
vRootDir = 'E:/q4/유형별 두피 이미지/Validation/'

vDirs = glob.glob(vRootDir+'**/*.json',recursive=True)
imgs =[]
labels = []
length = len(vDirs)
counter = 0
'''
for vDir in vDirs:
    counter = counter +1
    print(counter)
    length = length -1
    print("\r남은수 : ",length,'counter : ',counter,end='')
    with open(vDir,'r',encoding='UTF8') as file:
            data = json.load(file)
            #print(data)
            #print(data['value_1']+data['value_2']+data['value_3']+data['value_4']+data['value_5']+data['value_6'])
            encodedStatus = data['value_1']+data['value_2']+data['value_3']+data['value_4']+data['value_5']+data['value_6']
            labels.append(decodeStatus(encodedStatus))
        
            imgDir = (vDir.replace('라벨','원천')).replace('json','jpg')
            #print(imgDir)
            imgData = np.array(Image.open(imgDir).resize((width,height)).convert('RGB'))
            imgs.append(imgData)
            file.close()
imgs=np.array(imgs)
imgs=imgs/255
imgs=imgs.reshape(-1,width,height,3)
'''
def gen2():
    for lDir in vDirs:
        with open(lDir,'r',encoding='UTF8') as file:
                data = json.load(file)
                #print(data)
                #print(data['value_1']+data['value_2']+data['value_3']+data['value_4']+data['value_5']+data['value_6'])
                encodedStatus = data['value_1']+data['value_2']+data['value_3']+data['value_4']+data['value_5']+data['value_6']
                
        
                imgDir = (lDir.replace('라벨','원천')).replace('json','jpg')
                #print(imgDir)
                imgData = np.array(Image.open(imgDir).resize((width,height)).convert('RGB'))
                imgData = imgData/255 
               
            

                label = decodeStatus(encodedStatus)
                label=np.array(label)
                label=keras.utils.to_categorical(label,9)
                label=label.astype(np.int8)
               

                
                yield(imgData,label)
vdata = tf.data.Dataset.from_generator(gen2,(tf.int16,9),((width,height,3),(9)))
vdata = vdata.batch(700)

print('evaluate start')
score = loadedModel.evaluate(vdata)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
evScore




1.4406474828720093